In [3]:
from collections import Counter
import nltk
from nltk.tokenize import TweetTokenizer
import pandas as pd
import re as regex
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from time import time
import csv
import gensim
import preprocessor as p



In [4]:
dataset = pd.read_csv('source_replies_dataframe.csv', delimiter=",",quoting=csv.QUOTE_ALL, encoding="utf8")

In [5]:
dataset

,source_id,reply_id,topic,source_text,reply_text,nested,reply_type
0,5.002880e+17,5.002950e+17,ferguson,Did anyone think of comparing the clothes that...,Identical but irrelevant RT @TheAnonMessage2: ...,0,comment
1,5.002880e+17,5.002900e+17,ferguson,Did anyone think of comparing the clothes that...,"@TheAnonMessage2 no, because that would requir...",0,support
2,5.002880e+17,5.002900e+17,ferguson,Did anyone think of comparing the clothes that...,@TheAnonMessage2 http://t.co/nyf2SaMMh8,0,deny
3,5.002880e+17,5.002910e+17,ferguson,Did anyone think of comparing the clothes that...,"@RemarkablyDope @TheAnonMessage2 is it me, or ...",1,comment
4,5.002880e+17,5.002910e+17,ferguson,Did anyone think of comparing the clothes that...,@EricsSauces @TheAnonMessage2 it's not him lma...,1,comment
5,5.002880e+17,5.002910e+17,ferguson,Did anyone think of comparing the clothes that...,@RemarkablyDope @TheAnonMessage2 my God @stlco...,1,comment
6,5.002880e+17,5.002890e+17,ferguson,Did anyone think of comparing the clothes that...,.@TheAnonMessage2 yes. They don't match. Flip...,0,support
7,5.002880e+17,5.002890e+17,ferguson,Did anyone think of comparing the clothes that...,It's a good thing u went to detective school.....,0,comment
8,5.002880e+17,5.002890e+17,ferguson,Did anyone think of comparing the clothes that...,@TheAnonMessage2 yes hat and sandals vs no hat...,0,support
9,5.002880e+17,5.002900e+17,ferguson,Did anyone think of comparing the clothes that...,@Manning_Eli_1 @TheAnonMessage2 I thought the...,1,support


In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()


In [7]:
for index, row in dataset.iterrows():
    aa = row['reply_text']
   # for b in aa:
    snt = analyser.polarity_scores(aa)
    print (snt)

{'neg': 0.0, 'neu': 0.86, 'pos': 0.14, 'compound': 0.4767}
{'neg': 0.264, 'neu': 0.736, 'pos': 0.0, 'compound': -0.7003}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.893, 'pos': 0.107, 'compound': 0.34}
{'neg': 0.222, 'neu': 0.778, 'pos': 0.0, 'compound': -0.4847}
{'neg': 0.291, 'neu': 0.465, 'pos': 0.244, 'compound': -0.1027}
{'neg': 0.11, 'neu': 0.652, 'pos': 0.238, 'compound': 0.3687}
{'neg': 0.0, 'neu': 0.818, 'pos': 0.182, 'compound': 0.4404}
{'neg': 0.158, 'neu': 0.647, 'pos': 0.194, 'compound': 0.128}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.34}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.732, 'pos': 0.268, 'compound': 0.296}
{'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'compound': -0.1531}
{'neg': 0.359, 'neu': 0.573, 'pos': 0.068, 'compound': -0.765}
{'neg': 0.0, 'neu': 0.901, 'pos': 0.099, 'compound': 0.3182}
{'neg': 0.0, 'neu': 0.897, 'pos': 0.103, 

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.839, 'pos': 0.161, 'compound': 0.3612}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.526, 'pos': 0.474, 'compound': 0.4019}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.086, 'neu': 0.914, 'pos': 0.0, 'compound': -0.0572}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.26, 'neu': 0.505, 'pos': 0.235, 'compound': -0.09}
{'neg': 0.0, 'neu': 0.753, 'pos': 0.247, 'compound': 0.3182}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'compound': -0.5719}
{'neg': 0.549, 'neu': 0.451, 'pos': 0.0, 'compound': -0.8074}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.723, 'po

{'neg': 0.0, 'neu': 0.4, 'pos': 0.6, 'compound': 0.6696}
{'neg': 0.0, 'neu': 0.738, 'pos': 0.262, 'compound': 0.4926}
{'neg': 0.151, 'neu': 0.732, 'pos': 0.117, 'compound': -0.1779}
{'neg': 0.0, 'neu': 0.786, 'pos': 0.214, 'compound': 0.2914}
{'neg': 0.211, 'neu': 0.653, 'pos': 0.136, 'compound': -0.3612}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.159, 'neu': 0.718, 'pos': 0.123, 'compound': -0.1779}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.249, 'neu': 0.751, 'pos': 0.0, 'compound': -0.6486}
{'neg': 0.0, 'neu': 0.82, '

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.241, 'neu': 0.759, 'pos': 0.0, 'compound': -0.7184}
{'neg': 0.0, 'neu': 0.633, 'pos': 0.367, 'compound': 0.4404}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.653, 'pos': 0.347, 'compound': 0.4926}
{'neg': 0.466, 'neu': 0.376, 'pos': 0.158, 'compound': -0.6249}
{'neg': 0.0, 'neu': 0.263, 'pos': 0.737, 'compound': 0.4215}
{'neg': 0.0, 'neu': 0.484, 'pos': 0.516, 'compound': 0.4926}
{'neg': 0.162, 'neu': 0.741, 'pos': 0.098, 'compound': -0.2263}
{'neg': 0.224, 'neu': 0.621, 'pos': 0.155, 'compound': -0.2732}
{'neg': 0.091, 'neu': 0.909, 'pos': 0.0, 'compound': -0.34}
{'neg': 0.0, 'neu': 0.917, 'pos': 0.083, 'compound': 0.2023}
{'neg': 0.0, 'neu': 0.417, 'pos': 0.583, 'compound': 0.4215}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.226, 'neu': 0.774, 'pos': 0.0, 'compound

{'neg': 0.0, 'neu': 0.827, 'pos': 0.173, 'compound': 0.5178}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.492, 'neu': 0.508, 'pos': 0.0, 'compound': -0.4404}
{'neg': 0.252, 'neu': 0.445, 'pos': 0.303, 'compound': 0.1742}
{'neg': 0.303, 'neu': 0.697, 'pos': 0.0, 'compound': -0.8126}
{'neg': 0.521, 'neu': 0.479, 'pos': 0.0, 'compound': -0.8225}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.359, 'neu': 0.513, 'pos': 0.128, 'compound': -0.8481}
{'neg': 0.369, 'neu': 0.517, 'pos': 0.114, 'compound': -0.907}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.346, 'neu': 0.654, 'pos': 0.0, 'compound': -0.8126}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.739, 'pos': 0.261, 'compound': 0.5574}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 

{'neg': 0.212, 'neu': 0.788, 'pos': 0.0, 'compound': -0.5517}
{'neg': 0.5, 'neu': 0.5, 'pos': 0.0, 'compound': -0.9001}
{'neg': 0.23, 'neu': 0.67, 'pos': 0.1, 'compound': -0.4019}
{'neg': 0.0, 'neu': 0.435, 'pos': 0.565, 'compound': 0.6369}
{'neg': 0.0, 'neu': 0.847, 'pos': 0.153, 'compound': 0.4404}
{'neg': 0.298, 'neu': 0.702, 'pos': 0.0, 'compound': -0.5256}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.107, 'neu': 0.893, 'pos': 0.0, 'compound': -0.34}
{'neg': 0.279, 'neu': 0.721, 'pos': 0.0, 'compound': -0.7351}
{'neg': 0.0, 'neu': 0.588, 'pos': 0.412, 'compound': 0.4215}
{'neg': 0.133, 'neu': 0.556, 'pos': 0.311, 'compound': 0.4926}
{'neg': 0.315, 'neu': 0.685, 'pos': 0.0, 'compound': -0.7783}
{'neg': 0.0, 'neu': 0.828, 'pos': 0.172, 'compound': 0.3612}
{'neg': 0.086, 'neu': 0.914, 'pos': 0.0, 'compound': -0.2003}
{'neg': 0.136, 'neu': 0.864, 'pos': 0.0, 'compound': -0.296}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0,

In [8]:
snt = analyser.polarity_scores(aa)
print (snt)

{'neg': 0.163, 'neu': 0.837, 'pos': 0.0, 'compound': -0.5423}


In [9]:
keep_cols=['topic','source_text','reply_text','nested','reply_type']
newdata = dataset[keep_cols]

In [10]:
newdata

,topic,source_text,reply_text,nested,reply_type
0,ferguson,Did anyone think of comparing the clothes that...,Identical but irrelevant RT @TheAnonMessage2: ...,0,comment
1,ferguson,Did anyone think of comparing the clothes that...,"@TheAnonMessage2 no, because that would requir...",0,support
2,ferguson,Did anyone think of comparing the clothes that...,@TheAnonMessage2 http://t.co/nyf2SaMMh8,0,deny
3,ferguson,Did anyone think of comparing the clothes that...,"@RemarkablyDope @TheAnonMessage2 is it me, or ...",1,comment
4,ferguson,Did anyone think of comparing the clothes that...,@EricsSauces @TheAnonMessage2 it's not him lma...,1,comment
5,ferguson,Did anyone think of comparing the clothes that...,@RemarkablyDope @TheAnonMessage2 my God @stlco...,1,comment
6,ferguson,Did anyone think of comparing the clothes that...,.@TheAnonMessage2 yes. They don't match. Flip...,0,support
7,ferguson,Did anyone think of comparing the clothes that...,It's a good thing u went to detective school.....,0,comment
8,ferguson,Did anyone think of comparing the clothes that...,@TheAnonMessage2 yes hat and sandals vs no hat...,0,support
9,ferguson,Did anyone think of comparing the clothes that...,@Manning_Eli_1 @TheAnonMessage2 I thought the...,1,support


In [11]:
temp = [] 
for x in newdata['reply_text']:
    temp.append(p.clean(x))
temp = pd.Series(temp)
newdata['reply_text'] = temp
#print(newdata['reply_text'][:5])

g:\python 3.6.2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [12]:
newdata

,topic,source_text,reply_text,nested,reply_type
0,ferguson,Did anyone think of comparing the clothes that...,Identical but irrelevant RT : Did anyone think...,0,comment
1,ferguson,Did anyone think of comparing the clothes that...,"no, because that would require reasoning and t...",0,support
2,ferguson,Did anyone think of comparing the clothes that...,,0,deny
3,ferguson,Did anyone think of comparing the clothes that...,"is it me, or the dude in the security cam phot...",1,comment
4,ferguson,Did anyone think of comparing the clothes that...,it's not him lmao the store owners already sai...,1,comment
5,ferguson,Did anyone think of comparing the clothes that...,my God sucks,1,comment
6,ferguson,Did anyone think of comparing the clothes that...,. yes. They don't match. Flip flops + hat in s...,0,support
7,ferguson,Did anyone think of comparing the clothes that...,It's a good thing u went to detective school.....,0,comment
8,ferguson,Did anyone think of comparing the clothes that...,yes hat and sandals vs no hat and tennis shoes,0,support
9,ferguson,Did anyone think of comparing the clothes that...,I thought the same thing.,1,support


In [13]:
import json
def cuewordscount(countfile):
    fw = open("count.txt", "w")
    with open("cue_word_list.txt", "r") as f1:
        cue_words = []
        for line in f1:
            cue_words.append(line.strip())
        wordcount = dict((x, 0) for x in cue_words)
    
        for index, row in countfile.iterrows():
            tw = row["reply_text"]
            for w in re.findall(r"\w+", tw):
                if w in wordcount:
                    wordcount[w] += 1
            fw.write ("tweet" + str(index) + ":")
            fw.write (json.dumps(wordcount))
            fw.write ("\n")
            wordcount = dict((x, 0) for x in cue_words)
    fw.close()

In [14]:
keep_cols=['reply_text','reply_type']
tokenizedtw = newdata[keep_cols]

In [15]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

In [16]:
temp = [] 
for x in tokenizedtw['reply_text']:
    temp.append(tknzr.tokenize(x))
temp = pd.Series(temp)
tokenizedtw['reply_text'] = temp

g:\python 3.6.2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [198]:
tokenizedtw

,reply_text,reply_type
0,"[Identical, but, irrelevant, RT, :, Did, anyon...",comment
1,"[no, ,, because, that, would, require, reasoni...",support
2,[],deny
3,"[is, it, me, ,, or, the, dude, in, the, securi...",comment
4,"[it's, not, him, lmao, the, store, owners, alr...",comment
5,"[my, God, sucks]",comment
6,"[., yes, ., They, don't, match, ., Flip, flops...",support
7,"[It's, a, good, thing, u, went, to, detective,...",comment
8,"[yes, hat, and, sandals, vs, no, hat, and, ten...",support
9,"[I, thought, the, same, thing, .]",support


In [199]:
tokenizedtw['reply_text']

0       [Identical, but, irrelevant, RT, :, Did, anyon...
1       [no, ,, because, that, would, require, reasoni...
2                                                      []
3       [is, it, me, ,, or, the, dude, in, the, securi...
4       [it's, not, him, lmao, the, store, owners, alr...
5                                        [my, God, sucks]
6       [., yes, ., They, don't, match, ., Flip, flops...
7       [It's, a, good, thing, u, went, to, detective,...
8       [yes, hat, and, sandals, vs, no, hat, and, ten...
9                       [I, thought, the, same, thing, .]
10                   [had, the, same, flip, flops, on, .]
11      [Did, anyone, think, that, ,, holy, balls, ,, ...
12                       [Yeah, ., just, did, on, CNN, .]
13              [No, but, it, def, softens, the, blow, .]
14      [Even, if, he, had, robbed, a, convenience, st...
15      [it's, not, race, were, all, targeted, by, the...
16      [of, course, ,, however, please, do, not, try,...
17      [Good,

In [200]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [201]:
sentence = newdata['reply_text']

In [202]:
sentences = sentence.to_string()

In [203]:
result = ''.join([i for i in sentences if not i.isdigit()])
print (result)


       Identical but irrelevant RT : Did anyone think...
       no, because that would require reasoning and t...
                                                        
       is it me, or the dude in the security cam phot...
       it's not him lmao the store owners already sai...
                                            my God sucks
       . yes. They don't match. Flip flops + hat in s...
       It's a good thing u went to detective school.....
          yes hat and sandals vs no hat and tennis shoes
                               I thought the same thing.
                            had the same flip flops on.
      Did anyone think that, holy balls, he was old ...
                                 Yeah. just did on CNN.
                        No but it def softens the blow.
      Even if he had robbed a convenience store, tha...
      it's not race were all targeted by the governm...
      of course , however please do not try and comp...
              Good job, faggots. You I

In [204]:
keep_cols=['reply_text','reply_type']
tokenizedtw = newdata[keep_cols]

In [205]:
tokenizedtw

,reply_text,reply_type
0,Identical but irrelevant RT : Did anyone think...,comment
1,"no, because that would require reasoning and t...",support
2,,deny
3,"is it me, or the dude in the security cam phot...",comment
4,it's not him lmao the store owners already sai...,comment
5,my God sucks,comment
6,. yes. They don't match. Flip flops + hat in s...,support
7,It's a good thing u went to detective school.....,comment
8,yes hat and sandals vs no hat and tennis shoes,support
9,I thought the same thing.,support


In [206]:
tokenizedtw

,reply_text,reply_type
0,Identical but irrelevant RT : Did anyone think...,comment
1,"no, because that would require reasoning and t...",support
2,,deny
3,"is it me, or the dude in the security cam phot...",comment
4,it's not him lmao the store owners already sai...,comment
5,my God sucks,comment
6,. yes. They don't match. Flip flops + hat in s...,support
7,It's a good thing u went to detective school.....,comment
8,yes hat and sandals vs no hat and tennis shoes,support
9,I thought the same thing.,support


In [207]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [17]:
tokenizedtw.loc[tokenizedtw['reply_type']=='support','reply_type']= 0
#len(tokenizedtw[tokenizedtw.reply_type=='deny'])
#len(tokenizedtw[tokenizedtw.reply_type=='query'])
#len(tokenizedtw[tokenizedtw.reply_type=='comment'])


g:\python 3.6.2\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
g:\python 3.6.2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
tokenizedtw.loc[tokenizedtw['reply_type']=='comment','reply_type']= 1


g:\python 3.6.2\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
g:\python 3.6.2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
tokenizedtw.loc[tokenizedtw['reply_type']=='deny','reply_type']= 2

g:\python 3.6.2\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
g:\python 3.6.2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
tokenizedtw.loc[tokenizedtw['reply_type']=='query','reply_type']= 3

g:\python 3.6.2\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
g:\python 3.6.2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [212]:
tokenizedtw_x = tokenizedtw['reply_text']
tokenizedtw_y = tokenizedtw['reply_type']

In [213]:
result.replace("\n","")


'       Identical but irrelevant RT : Did anyone think...       no, because that would require reasoning and t...                                                               is it me, or the dude in the security cam phot...       it\'s not him lmao the store owners already sai...                                            my God sucks       . yes. They don\'t match. Flip flops + hat in s...       It\'s a good thing u went to detective school.....          yes hat and sandals vs no hat and tennis shoes                               I thought the same thing.                            had the same flip flops on.      Did anyone think that, holy balls, he was old ...                                 Yeah. just did on CNN.                        No but it def softens the blow.      Even if he had robbed a convenience store, tha...      it\'s not race were all targeted by the governm...      of course , however please do not try and comp...              Good job, faggots. You IDd the wrong

In [214]:
update1 = result.replace("\t", "").replace("\n","")

In [215]:
new = update1.strip()

In [216]:
newsplit = new.split()

In [217]:
newsplit

['Identical',
 'but',
 'irrelevant',
 'RT',
 ':',
 'Did',
 'anyone',
 'think...',
 'no,',
 'because',
 'that',
 'would',
 'require',
 'reasoning',
 'and',
 't...',
 'is',
 'it',
 'me,',
 'or',
 'the',
 'dude',
 'in',
 'the',
 'security',
 'cam',
 'phot...',
 "it's",
 'not',
 'him',
 'lmao',
 'the',
 'store',
 'owners',
 'already',
 'sai...',
 'my',
 'God',
 'sucks',
 '.',
 'yes.',
 'They',
 "don't",
 'match.',
 'Flip',
 'flops',
 '+',
 'hat',
 'in',
 's...',
 "It's",
 'a',
 'good',
 'thing',
 'u',
 'went',
 'to',
 'detective',
 'school.....',
 'yes',
 'hat',
 'and',
 'sandals',
 'vs',
 'no',
 'hat',
 'and',
 'tennis',
 'shoes',
 'I',
 'thought',
 'the',
 'same',
 'thing.',
 'had',
 'the',
 'same',
 'flip',
 'flops',
 'on.',
 'Did',
 'anyone',
 'think',
 'that,',
 'holy',
 'balls,',
 'he',
 'was',
 'old',
 '...',
 'Yeah.',
 'just',
 'did',
 'on',
 'CNN.',
 'No',
 'but',
 'it',
 'def',
 'softens',
 'the',
 'blow.',
 'Even',
 'if',
 'he',
 'had',
 'robbed',
 'a',
 'convenience',
 'store,'

In [218]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [219]:
cv = TfidfVectorizer(min_df=1, stop_words='english')

In [220]:
x_train, x_test, y_train, y_test = train_test_split(tokenizedtw_x, tokenizedtw_y, test_size= 0.2, random_state=4)

In [257]:
y_test

1307    0
1687    2
615     1
3588    1
2300    1
2017    1
3470    1
4009    1
947     1
1779    0
1282    2
4035    1
2121    1
3828    1
681     1
2120    1
1823    1
491     1
3955    1
3518    2
3476    0
614     1
203     1
1321    0
515     1
325     1
2281    1
216     1
1497    3
2038    1
       ..
2303    1
3843    1
295     1
2323    1
222     1
922     1
3418    1
495     1
1302    1
2252    1
2925    1
2678    1
2667    3
115     1
2596    0
3665    1
1154    1
1263    1
1048    1
3255    2
1714    2
3647    1
649     0
686     1
1797    1
3269    1
1780    1
370     1
3636    1
2079    1
Name: reply_type, Length: 845, dtype: object

In [221]:
x_train.head()

2666    It appears Thelma &amp; Louise have rocket lau...
1705    Did you see him stand 30' away and get shot? N...
141                              Didn't I say "FLAKE OFF"
667     what? ": In 2009, police beat a man and charge...
3601      thanks so much for sharing. Just added it here:
Name: reply_text, dtype: object

In [222]:
cv = TfidfVectorizer(min_df=1,stop_words='english')

In [223]:
x_traincv = cv.fit_transform(newsplit)

In [224]:
x_traincv.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [225]:
cv.get_feature_names()

['aaaaand',
 'abbott',
 'abc',
 'abetted',
 'abiding',
 'abl',
 'able',
 'aboli',
 'aborted',
 'abou',
 'absence',
 'absolut',
 'absolute',
 'absolutely',
 'absolutly',
 'absorbed',
 'abt',
 'abu',
 'abuse',
 'abusing',
 'abysmal',
 'ac',
 'acc',
 'accelerate',
 'accept',
 'accepted',
 'access',
 'accident',
 'acco',
 'according',
 'account',
 'accounts',
 'accross',
 'accurate',
 'accuse',
 'achieved',
 'acknowledge',
 'act',
 'acted',
 'acting',
 'action',
 'actions',
 'active',
 'actor',
 'actress',
 'acts',
 'actu',
 'actual',
 'actually',
 'ad',
 'adam',
 'add',
 'added',
 'adding',
 'address',
 'adhering',
 'adjective',
 'administered',
 'administr',
 'admired',
 'admit',
 'admits',
 'admitted',
 'admitting',
 'adult',
 'affable',
 'affected',
 'affects',
 'affluent',
 'afghan',
 'afghans',
 'afp',
 'afraid',
 'africa',
 'african',
 'aft',
 'afternoon',
 'aftr',
 'aga',
 'age',
 'agenda',
 'agents',
 'aggressive',
 'ago',
 'agony',
 'agre',
 'agree',
 'agreed',
 'agrees',
 'ah',


In [226]:
cv1 = TfidfVectorizer(min_df=1,stop_words='english')

In [227]:
x_traincv=cv1.fit_transform(x_train)

In [228]:
a= x_traincv.toarray()

In [229]:
a

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [230]:
cv1.inverse_transform(a[3200])

[array(['holy', 'shit'],
       dtype='<U41')]

In [231]:
x_testcv=cv1.transform(x_test)

In [232]:
x_testcv= cv1.transform(x_test)

In [233]:
x_testcv.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [234]:
y_train=y_train.astype('int')

In [235]:
y_train

2666    1
1705    2
141     1
667     1
3601    1
3392    1
983     1
3035    1
3420    1
1600    0
671     1
2724    1
2980    1
254     1
2381    2
3999    1
2625    2
3118    1
3358    1
2503    1
934     1
3027    1
834     1
81      2
3262    1
335     2
2421    1
3028    1
1168    3
3268    1
       ..
1977    1
2143    1
1743    1
3442    1
1662    1
680     1
1440    2
3002    1
3394    3
3633    1
3720    1
1054    1
56      1
2470    1
3785    1
149     1
2581    3
1024    1
1411    1
1588    3
946     1
2871    1
3678    0
1082    2
456     0
3671    1
709     0
2487    0
174     1
1146    1
Name: reply_type, Length: 3376, dtype: int32

In [238]:
params = {'n_estimators': 1200, 'max_depth': 9, 'subsample': 0.8,
'learning_rate': 0.01, 'min_samples_leaf': 1, 'random_state': 3}

In [239]:
from sklearn import ensemble
clf = ensemble.GradientBoostingClassifier(**params)


In [240]:
clf

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=9,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1200,
              presort='auto', random_state=3, subsample=0.8, verbose=0,
              warm_start=False)

In [241]:
clf.fit(x_traincv, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=9,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1200,
              presort='auto', random_state=3, subsample=0.8, verbose=0,
              warm_start=False)

In [242]:
y_pred = clf.predict(x_testcv)


In [272]:
len (y_pred)
count = 0

845

In [282]:
y_true= np.array(y_test)

In [284]:
type (y_true)

numpy.ndarray

In [285]:
for i in range (len(y_pred)):
    if y_pred[i] == y_true[i]:
        count+= 1
        
    

In [286]:
count

563

In [288]:
acc = count/ len(y_pred)
acc

0.6662721893491125

In [289]:
#acc = clf.score(y_pred, y_test)

In [ ]:
#ms.accuracy_score(y_test, predictions)

In [ ]:
#a=np.array(y_test)

In [ ]:
#for i in range(len(predictions)):
 #   if(predictions[i]!=a[i]):
  #      count = count+1
   #     print(count)

In [ ]:
#count

In [ ]:
#good_words

In [ ]:
#model = Word2Vec(tokens, size=100, window=5, min_count=5, workers=4)

In [ ]:
#from gensim.models.keyedvectors import KeyedVectors
#model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz',binary=True,unicode_errors='ignore')

In [ ]:
#vectors =[model[w] for w in tokenizedList]

In [ ]:
#try: 
 #   c = model['of']
#except KeyError:
 #   c = 0

In [ ]:
#try: 
    #vectors = [model[w] for w in tokenizedList]
#except KeyError:
    #pass

In [ ]:
#import os

In [ ]:
#vectors

In [ ]:
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#analyser= SentimentIntensityAnalyzer()

In [ ]:
#def print_sentiment_scores(sentence):
 #   snt = analyser.polarity_scores(sentence)
  #  print("{:-<40} {}".format(sentence, str(snt)))